In [18]:
%pip install -q python-dotenv openai


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
import dotenv
dotenv.load_dotenv()

False

In [20]:
system_prompt = '''
You are an AI assistant who can perform the following steps:
1. Reason through the problem by describing your thoughts in a "Thought:" section.
2. When you need to use a tool, output an "Action:" section with the tool name and its input.
3. After the tool call, you'll see an "Observation:" section with the tool's output.
4. Continue this cycle of Thought → Action → Observation as needed.
5. End with a concise "Final Answer:" that answers the user's query.

Note:
- The chain of thought in "Thought:" sections is only visible to you and not part of your final answer.
- The user should only see your "Final Answer:".
'''

In [21]:
user_prompt = '''
What is the weather in Thunder Bay, Ontario, Canada Today?
'''

In [28]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user","content": user_prompt}
    ]
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [27]:
text = completion.choices[0].message.content
print(text)


NameError: name 'completion' is not defined

In [26]:
import re
pattern = r'Action:\s*(\w+)\("([^"]+)"\)'

match = re.search(pattern, text)
if match:
    tool_name = match.group(1)    # 'GetWeather'
    tool_input = match.group(2)   # 'Thunder Bay, Ontario, Canada'
    print("Tool name:", tool_name)
    print("Tool input:", tool_input)
else:
    print("No match found.")

NameError: name 'text' is not defined

In [24]:
import requests
import os

def get_current_weather(city_name):
    #base_url = "https://api.openweathermap.org/data/3.0/onecall"
    #params = {
    #    "lat": 48.3809,
    #    "lon": 89.2477,
    #    "appid": os.environ.get('OPENWEATHERMAPS_API_KEY'),
    #    "units": "metric"  # use "imperial" for Fahrenheit
    #}

    # Make the GET request
    #response = requests.get(base_url, params=params)
    
    # Raise an exception if there's an HTTP error
    #response.raise_for_status()
    
    # Parse the JSON response
    #data = response.json()

    # Extract relevant fields from the response
    #weather_info = {
    #    "city": data["name"],
    #    "temperature": data["main"]["temp"],
    #    "description": data["weather"][0]["description"],
    #    "humidity": data["main"]["humidity"]
    #}
    weather_info = {
        "city": "Thunder Bay",
        "temperature": -5.2,   # in Celsius
        "description": "snow",
        "humidity": 85         # in percentage
    }   
    return weather_info

In [25]:
if tool_name == 'GetWeather':
    weather_info = get_current_weather(tool_input)
    print(weather_info)
{'city': 'Thunder Bay', 'temperature': -5.2, 'description': 'snow', 'humidity': 85}

NameError: name 'tool_name' is not defined

In [ ]:
updated_text = text + f"\n\n Observation: {weather_info}"
print(updated_text)

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user","content": user_prompt},
        {"role": "assistant","content": text},
        {"role": "user","content": updated_text}
    ]

In [ ]:
text2 = completion.choices[0].message.content
print(text2)